In [1]:
!pip install datasets

In [2]:
!pip install transformers

In [3]:
!pip install accelerate

In [4]:
!pip install fuzzywuzzy

In [5]:
!pip install python-Levenshtein

In [8]:
!pip install gensim

   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/24.0 MB ? eta -:--:--
    --------------------------------------- 0.5/24.0 MB 762.0 kB/s eta 0:00:31
   - -------------------------------------- 0.8/24.0 MB 799.2 kB/s eta 0:00:30
   - -------------------------------------- 1.0/24.0 MB 986.7 kB/s eta 0:00:24
   - -------------------------------------- 1.0/24.0 MB 986.7 kB/s eta 0:00:24
   -- ------------------------------------- 1.3/24.0 MB 944.7 kB/s eta 0:00:25
   --- ------------------------------------ 1.8/24.0 MB 1.1 MB/s eta 0:00:21
   --- ------------------------------------ 1.8/24.0 MB 1.1 MB/s eta 0:00:21
   --- ------------------------------------ 2.1/24.0 MB 1.1 MB/s eta 0:00:21
   --- ------------------------------------ 2.4/24.0 MB 1.0 MB/s eta 0:00:22
   --- ----------

In [9]:
!pip install nltk

In [21]:
import random
from typing import Dict, List, Union
import random
from typing import Dict, List, Union
from datasets import load_dataset
from fuzzywuzzy import fuzz, process
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm
from transformers import pipeline
from gensim.models import Word2Vec
import nltk
from nltk.corpus import wordnet
from nltk import word_tokenize
from datasets import Dataset, concatenate_datasets

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\snytk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\snytk\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\snytk\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [11]:
def augment_text(text: str) -> str: # не очень
    words = word_tokenize(text)
    augmented_words = []
    for word in words:
        synonyms = wordnet.synsets(word)
        if synonyms:
            synonym = synonyms[0].lemmas()[0].name()
            augmented_words.append(synonym.replace('_', ' '))
        else:
            augmented_words.append(word)
    return ' '.join(augmented_words)

def prepare_message_for_llm(text: Union[str, List[str]], categories: Dict[str, str]) -> Dict[str, Union[List[Dict[str, str]], List[List[Dict[str, str]]]]]:
    if len(categories) < 2:
        raise RuntimeError(f'The category list is too small! Expected 2 or more categories, got {len(categories)} ones.')
    categories_ = sorted(list(categories.keys()))
    categories_as_string = ', '.join(categories_[:-1]) + ' и ' + categories_[-1]
    if isinstance(text, str):
        prompt = f'Прочтите, пожалуйста, следующий текст и определите, какая тема из известного ' \
                 f'списка тем наиболее представлена в следующем тексте. ' \
                 f'В качестве ответа напишите только название темы из списка, больше ничего.\n' \
                 f'Список тем: {categories_as_string}.\n'
        for cur in categories_:
            prompt += f'Текст: {" ".join(categories[cur].split())}\nВаш ответ: {cur}\n'
        prompt += f'Текст: {" ".join(text.split())}\nВаш ответ: '
        messages = [
            {
                'role': 'system',
                'content': 'Вы - полезный помощник, умеющий читать тексты на русском языке, глубоко понимать их и анализировать.'
            },
            {
                'role': 'user',
                'content': prompt
            }
        ]
    else:
        messages = []
        for it in text:
            prompt = f'Прочтите, пожалуйста, следующий текст и определите, какая тема из известного ' \
                     f'списка тем наиболее представлена в следующем тексте. ' \
                     f'В качестве ответа напишите только название темы из списка, больше ничего.\n' \
                     f'Список тем: {categories_as_string}.\n'
            for cur in categories_:
                prompt += f'Текст: {" ".join(categories[cur].split())}\nВаш ответ: {cur}\n'
            prompt += f'Текст: {" ".join(it.split())}\nВаш ответ: '
            messages.append([
                {
                    'role': 'system',
                    'content': 'Вы - полезный помощник, умеющий читать тексты на русском языке, глубоко понимать их и анализировать.'
                },
                {
                    'role': 'user',
                    'content': prompt
                }
            ])
    return {'message_for_llm': messages}

In [12]:
llm_pipeline = pipeline(model='Qwen/Qwen2-7B-Instruct', device_map='auto', torch_dtype='auto')

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

C:\Users\snytk\miniconda3\envs\python311\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\snytk\.cache\huggingface\hub\models--Qwen--Qwen2-7B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

Some parameters are on the meta device because they were offloaded to the disk and cpu.


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [13]:
DATASET_NAME = 'Davlan/sib200'
DATASET_LANGUAGE = 'rus_Cyrl'
train_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='train')
validation_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='validation')
test_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='test')

In [14]:
list_of_categories = sorted(list(
    set(train_set['category']) | set(validation_set['category']) | set(test_set['category'])
))
print(f'Categories for classification are: {list_of_categories}')

Categories for classification are: ['entertainment', 'geography', 'health', 'politics', 'science/technology', 'sports', 'travel']


In [22]:
augmented_data = []
nltk.data.path.append(r"C:\Users\snytk\AppData\Roaming\nltk_data")

for example in train_set:
    augmented_text = augment_text(example['text'])
    augmented_data.append({'text': augmented_text, 'category': example['category']})

augmented_dataset = Dataset.from_list(augmented_data)

train_set = concatenate_datasets([train_set, augmented_dataset])

examples_by_categories = dict()
for current_category in list_of_categories:
    examples_by_categories[current_category] = random.choice(train_set.filter(lambda it: it['category'] == current_category)['text'])

print(validation_set)

Filter:   0%|          | 0/1402 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1402 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1402 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1402 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1402 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1402 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1402 [00:00<?, ? examples/s]

Dataset({
    features: ['index_id', 'category', 'text'],
    num_rows: 99
})


In [23]:
validation_set_for_llm = validation_set.map(lambda it: prepare_message_for_llm(it['text'], examples_by_categories))
test_set_for_llm = test_set.map(lambda it: prepare_message_for_llm(it['text'], examples_by_categories))

print(validation_set_for_llm)

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Map:   0%|          | 0/204 [00:00<?, ? examples/s]

Dataset({
    features: ['index_id', 'category', 'text', 'message_for_llm'],
    num_rows: 99
})


In [24]:
y_pred = list(map(
    lambda x: llm_pipeline(x, max_new_tokens=10)[0]['generated_text'],
    tqdm(validation_set_for_llm['message_for_llm'])
))
y_true = validation_set['category']

print(classification_report(y_true=y_true, y_pred=[x[-1]['content'] for x in y_pred]))

  0%|          | 0/99 [00:00<?, ?it/s]

C:\Users\snytk\miniconda3\envs\python311\Lib\site-packages\transformers\models\qwen2\modeling_qwen2.py:544: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


                    precision    recall  f1-score   support

           culture       0.00      0.00      0.00         0
     entertainment       0.56      0.56      0.56         9
         geography       0.86      0.75      0.80         8
            health       0.90      0.82      0.86        11
           history       0.00      0.00      0.00         0
          politics       0.87      0.93      0.90        14
science/technology       0.88      0.84      0.86        25
            sports       0.92      1.00      0.96        12
           traffic       0.00      0.00      0.00         0
         transport       0.00      0.00      0.00         0
            travel       0.75      0.60      0.67        20
          wildlife       0.00      0.00      0.00         0

          accuracy                           0.79        99
         macro avg       0.48      0.46      0.47        99
      weighted avg       0.83      0.79      0.80        99



C:\Users\snytk\miniconda3\envs\python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\snytk\miniconda3\envs\python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\snytk\miniconda3\envs\python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [25]:
y_pred_with_normalization = list(map(
    lambda it: process.extractOne(it[-1]['content'], list_of_categories, scorer=fuzz.token_sort_ratio)[0],
    y_pred
))

print(classification_report(y_true=y_true, y_pred=y_pred_with_normalization))

                    precision    recall  f1-score   support

     entertainment       0.56      0.56      0.56         9
         geography       0.86      0.75      0.80         8
            health       0.90      0.82      0.86        11
          politics       0.81      0.93      0.87        14
science/technology       0.88      0.84      0.86        25
            sports       0.80      1.00      0.89        12
            travel       0.72      0.65      0.68        20

          accuracy                           0.80        99
         macro avg       0.79      0.79      0.79        99
      weighted avg       0.80      0.80      0.80        99



In [26]:
# Тестирование на тестовом наборе
y_pred = list(map(
    lambda x: llm_pipeline(x, max_new_tokens=10)[0]['generated_text'],
    tqdm(test_set_for_llm['message_for_llm'])
))
y_true = test_set['category']

print(classification_report(y_true=y_true, y_pred=[x[-1]['content'] for x in y_pred]))

  0%|          | 0/204 [00:00<?, ?it/s]

                    precision    recall  f1-score   support

           animals       0.00      0.00      0.00         0
               art       0.00      0.00      0.00         0
         astronomy       0.00      0.00      0.00         0
     entertainment       0.80      0.84      0.82        19
         geography       0.93      0.76      0.84        17
            health       0.95      0.82      0.88        22
           history       0.00      0.00      0.00         0
        literature       0.00      0.00      0.00         0
             music       0.00      0.00      0.00         0
          politics       0.91      0.97      0.94        30
science/technology       0.88      0.88      0.88        51
            sports       0.82      0.92      0.87        25
        technology       0.00      0.00      0.00         0
         transport       0.00      0.00      0.00         0
    transportation       0.00      0.00      0.00         0
            travel       1.00      0.68

C:\Users\snytk\miniconda3\envs\python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\snytk\miniconda3\envs\python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\snytk\miniconda3\envs\python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [27]:
y_pred_with_normalization = list(map(
    lambda it: process.extractOne(it[-1]['content'], list_of_categories, scorer=fuzz.token_sort_ratio)[0],
    y_pred
))

print(classification_report(y_true=y_true, y_pred=y_pred_with_normalization))

                    precision    recall  f1-score   support

     entertainment       0.80      0.84      0.82        19
         geography       0.93      0.76      0.84        17
            health       0.78      0.82      0.80        22
          politics       0.85      0.97      0.91        30
science/technology       0.87      0.90      0.88        51
            sports       0.72      0.92      0.81        25
            travel       0.96      0.68      0.79        40

          accuracy                           0.84       204
         macro avg       0.85      0.84      0.84       204
      weighted avg       0.86      0.84      0.84       204

